In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
import tensorflow as tf
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential
import keras 
from keras.layers import Dense, Flatten, Conv1D,MaxPooling1D, Dropout,BatchNormalization,Embedding,Concatenate, Activation,Input,Bidirectional,LSTM
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras import backend as K


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

<h2> Basic data cooking

<h3> Data for OneHot Models

In [ ]:
!pip install git+https://github.com/Lpourchot/dfencoding.git

In [ ]:
from dfencoding import utilities

In [ ]:
train_dum = train.copy()
test_dum = test.copy()

In [ ]:
train_dum = train_dum.iloc[:,1:].astype('str')
test_dum = test_dum.iloc[:,1:].astype('str')

In [ ]:
dfe = utilities.dfencoding(train_dum,'target',test_dum, missing_value = 'Y', cat_limit = 150, dummies_limit = 150)

In [ ]:
dfe.get_dummies()

In [ ]:
X_Onehot = dfe.data.iloc[:len(train_dum),1:]
test_Onehot = dfe.data.iloc[len(train_dum):,1:]
print(X_Onehot.shape)
print(test_Onehot.shape)

<h3> Data for others Models

In [ ]:
# Without labelencoding
target = pd.get_dummies(train['target'])
y = train['target']
X = train.iloc[:,1:-1]
test = test.iloc[:,1:]

# To avoid negative values (for embedding), we just add 8 to all categories :
X = X + 8
test = test + 8
X.shape, test.shape, y.shape, target.shape

In [ ]:
target = pd.get_dummies(train['target']).astype('float')

<h3> Parameters for the training

In [ ]:
es = callbacks.EarlyStopping(
                monitor = 'val_categorical_crossentropy', 
                min_delta = 0.0000001, 
                patience = 4,
                mode = 'min',
                baseline = None, 
                restore_best_weights = True,
                verbose = 1)

plateau  = callbacks.ReduceLROnPlateau(
                monitor = 'val_categorical_crossentropy',
                factor = 0.5, 
                patience = 2, 
                mode = 'min', 
                min_delt = 0.0000001,
                cooldown = 0, 
                min_lr = 1e-7,
                verbose = 1) 

metrics = [tf.keras.metrics.CategoricalCrossentropy()]
loss = tf.keras.losses.CategoricalCrossentropy(
                from_logits=False,
                label_smoothing=0,
                reduction="auto",
                name="categorical_crossentropy")


<h2> Base models : Decision Forest + Embedding + Conv1D + Sequential Onehot

<h3> Decision Forest Model

<h4> Class Decision tree (they are Base components of the forest)

In [ ]:
"""
Adapted from : 
https://keras.io/examples/structured_data/deep_neural_decision_forests/
If you try the source from Keras, I found an issue in the source : 
change vocabulary_size by vocab_size in the data pipeline (encode function)
"""
class Decision_Tree(keras.Model):
    def __init__(self, depth, num_features, used_features_rate, num_classes):
        super(Decision_Tree, self).__init__()
        self.depth = depth
        self.num_leaves = 2 ** depth
        self.num_classes = num_classes
        num_used_features = int(num_features * used_features_rate)    
        one_hot = np.eye(num_features)                                
        sampled_feature_indicies = np.random.choice(
            np.arange(num_features), num_used_features, replace=False
        )                                                            
        self.used_features_mask = one_hot[sampled_feature_indicies]   
        self.pi = tf.Variable(
            initial_value = tf.random_normal_initializer()(
            shape = [self.num_leaves, self.num_classes]
            ),
            dtype="float32",
            trainable=True,
        )
        
        self.decision_fn = layers.Dense(
            units=self.num_leaves, 
            activation="sigmoid", 
            name="decision"
            )

    def call(self, features):
        batch_size = tf.shape(features)[0]
        features = tf.matmul(
            features, 
            self.used_features_mask, 
            transpose_b=True
            )  
        decisions = tf.expand_dims(
            self.decision_fn(features),
            axis=2
            )  
        decisions = layers.concatenate(
            [decisions, 1 - decisions],
            axis=2
            ) 

        mu = tf.ones([batch_size, 1, 1]) 
        begin_idx = 1
        end_idx = 2

        for level in range(self.depth):
            mu = tf.reshape(mu, [batch_size, -1, 1])  
            mu = tf.tile(mu, (1, 1, 2))  
            level_decisions = decisions[:, begin_idx:end_idx, :]  
            mu = mu * level_decisions  
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (level + 1)

        mu = tf.reshape(mu, [batch_size, self.num_leaves])  
        probabilities = keras.activations.softmax(self.pi)  
        outputs = tf.matmul(mu, probabilities) 
        
        return outputs

<h4> Class Decision Forest

In [ ]:
class Decision_Forest(keras.Model):
    def __init__(self, num_trees, depth, num_features, used_features_rate, num_classes):
        super(Decision_Forest, self).__init__()
        self.ensemble = []
        self.num_classes = num_classes
        
        for _ in range(num_trees):
            self.ensemble.append(
                            Decision_Tree(depth, 
                            num_features,
                            used_features_rate,
                            self.num_classes)
                            )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        outputs = tf.zeros([batch_size, 
                            num_classes])
        
        for tree in self.ensemble:
            outputs += tree(inputs)
            
        outputs /= len(self.ensemble)
        
        return outputs

<h4> Decision Forest Model ( Decision_Forest (Decision_Tree) )

In [ ]:
def api_func_forest():
    num_trees = 20
    depth = 5
    used_features_rate = 0.5
    num_classes = 4
    num_features = 50
    API_input_forest = Input(shape = (50,),
                            name = 'API_input_forest')
    num_features = 50
    forest_model = Decision_Forest(
                            num_trees,
                            depth, 
                            num_features,
                            used_features_rate,
                            num_classes
                            )

    API_output_forest = forest_model(API_input_forest)

    return API_output_forest, API_input_forest

num_classes = 4

<h3> OneHot Model

In [ ]:
def api_func_onhot():
    inputs_API_Onehot = Input(shape = (1285,), name = 'API_input_Onehot')
    w = Dense(1274, activation="relu")(inputs_API_Onehot)
    w = Dropout(0.3)(w)
    w = Dense(20, activation = "relu")(w)
    w = Dropout(0.3)(w)
    outputs_API_Onehot = Dense(4, activation = "relu")(w)
    
    return  outputs_API_Onehot,inputs_API_Onehot

<h3> Embedding Model

In [ ]:
def api_func_embedding():
    inputs_API_Embedding = Input(shape = (50,), name = 'API_input_Embedding')
    x = Embedding(80, 10, input_length = 50)(inputs_API_Embedding)
    x = Flatten()(x)
    x = Dense(80, activation = "relu")(x)
    x = Dense(20, activation = 'relu')(x)
    outputs_API_Embedding = Dense(4, activation='relu')(x)
    
    return outputs_API_Embedding,inputs_API_Embedding

<h3> Conv1D Model

In [ ]:
def api_func_conv1D():
    inputs_API_Conv1D = Input(shape=(50,1), name = 'API_input_Conv1D') 
    v = Conv1D(
                filters = 256, 
                kernel_size = 4,
                padding = 'same', 
                activation = 'relu',
                )(inputs_API_Conv1D)

    v = MaxPooling1D(pool_size = 3)(v)
    v = Flatten()(v)
    v = Dense(64, activation ='relu')(v)
    outputs_API_Conv1D = Dense(4, activation = 'relu')(v)
    
    return outputs_API_Conv1D,inputs_API_Conv1D

<h2>Decision Forest + Conv1D + Embedding + Onehot

In [ ]:
N_FOLDS = 10
SEED = 2021
oof = np.zeros((X.shape[0],4))
pred = np.zeros((test.shape[0],4))
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, ts_idx) in enumerate(skf.split(X, y)):
    print(f"===== FOLD {fold} =====")
       
    x_tr = X.iloc[tr_idx] 
    x_Onehot_tr = X_Onehot.iloc[tr_idx] 
    y_tr = target.iloc[tr_idx] 
    x_ts = X.iloc[ts_idx] 
    x_Onehot_ts = X_Onehot.iloc[ts_idx] 
    y_ts = target.iloc[ts_idx] 
      
    #---------- Base models collection ---------------------
    outputs_API_Onehot, inputs_API_Onehot = api_func_onhot()
    outputs_API_Embedding, inputs_API_Embedding = api_func_embedding()    
    outputs_API_Conv1D, inputs_API_Conv1D = api_func_conv1D()
    API_output_forest, API_input_forest = api_func_forest()
    
   #---------- Final Model Layers --------------------------  
    z = Concatenate(axis=1)(
            [outputs_API_Conv1D,
            outputs_API_Embedding,
            outputs_API_Onehot,
            API_output_forest])
    z = Dense(16, activation = 'sigmoid')(z)
    out = Dense(4, activation = 'softmax', name = 'out')(z)
    
    #----------Model instantiation--------------------------- 
    model_merged = Model(
            inputs=[inputs_API_Conv1D,
            inputs_API_Embedding,
            inputs_API_Onehot,
            API_input_forest], 
            outputs=out, 
            name="model_merged")
        
    #----------Model compile--------------------------- 
    model_merged.compile(
            tf.keras.optimizers.Adam(learning_rate=0.001),
            loss = loss ,
            metrics = metrics)
    
    #----------Model fit--------------------------- 
    model_merged.fit(
            {'API_input_Conv1D': x_tr,
            'API_input_Embedding': x_tr,
            'API_input_Onehot': x_Onehot_tr,
            'API_input_forest': x_tr},
            {'out': y_tr},
            validation_data = ([x_ts,x_ts,x_Onehot_ts,x_ts], y_ts),
            batch_size = 256,
            epochs = 50,
            verbose = 1,
            callbacks = [es,plateau])
    
    oof[ts_idx] = model_merged.predict(
            [x_ts,x_ts, 
             x_Onehot_ts, 
             x_ts])
    
    score = log_loss(y_ts, oof[ts_idx])
    print(f"FOLD {fold} Score {score}\n")
    
    pred += model_merged.predict([test, test, test_Onehot, test]) / N_FOLDS

score = log_loss(y, oof)
print(f"Score total {score}\n")   

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
submission_df = pd.DataFrame(pred)
submission_df.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4']
submission_df['id'] = submission['id']
submission_df = submission_df[['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4']]
submission_df.to_csv("submission_Keras_10.csv", index=False)
display(submission_df.head())